In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.cluster import KMeans

sys.path.append("/Users/nwong/Workspace/Projects/tdt4173_project/src")

from feature_engineering.sklearn_transformers import *
from feature_engineering.store_features import *
from feature_engineering.bus_stop_features import *
from feature_engineering.utils import *
import h2o
from h2o.automl import H2OAutoML

stop_importance_levels = ['Mangler viktighetsnivå',
                          'Standard holdeplass',
                          'Lokalt knutepunkt',
                          'Nasjonalt knutepunkt',
                          'Regionalt knutepunkt',
                          'Annen viktig holdeplass']
store_types = ['lv1_desc', 'lv2_desc', 'lv3_desc', 'lv4_desc']
geo_groups = ['grunnkrets_id','t_district','municipality_name']

In [2]:
stores_train_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_train.csv"))
stores_test_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_test.csv"))
stores_extra_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_extra.csv"))

income_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_income_households.csv"))
age_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_age_distribution.csv"))
household_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_households_num_persons.csv"))
grunnkrets_df = combine_keys(set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_norway_stripped.csv")))
plaace_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/plaace_hierarchy.csv")

bus_stops_df = bus_stops_lat_lon(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/busstops_norway.csv"))


/Users/nwong/Workspace/Projects/tdt4173_project/src/feature_engineering/bus_stop_features.py:11: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  bus_stops_df[['lon', 'lat']] = bus_stops_df['lng_lat'].str.split(


In [3]:
stores_train_merged = encode_levels(stores_train_df.merge(plaace_df, on="plaace_hierarchy_id", how="left"))
stores_test_merged = encode_levels(stores_test_df.merge(plaace_df, on="plaace_hierarchy_id", how="left"))
stores_extra_merged = encode_levels(stores_extra_df.merge(plaace_df, on="plaace_hierarchy_id", how="left"))

In [4]:
compare_train_df = pd.concat([stores_train_merged, stores_extra_merged], ignore_index=True)
compare_test_df = pd.concat([stores_test_merged, stores_extra_merged], ignore_index=True)

In [5]:
def stores_in_radius_new(stores_merged, compare_df, radius=0.05):
    lv_1 = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group='lv1_desc')
    lv_2 = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group='lv2_desc')
    lv_3 = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group='lv3_desc')
    lv_4 = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group='lv4_desc')
    all_count = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group=None)
    
    return lv_1.merge(lv_2, on="store_id", how="inner").merge(lv_3, on="store_id", how="inner").merge(lv_4, on="store_id", how="inner").merge(all_count, on="store_id", how="inner")

In [6]:
def new_pop_density(stores_df, age_dist, grunnkrets_df, geo_groups):
    population_density = population_density_grouped_by_geo_group(stores_df, age_dist, grunnkrets_df, geo_groups)
    return population_density.fillna(population_density.mean())

In [7]:

def new_age_dist(stores_df, age_df, grunnkrets_df, geo_groups):
    _age_dist = age_dist_by_geo_group(stores_train_df, age_dist, grunnkrets_df, geo_groups)
    return _age_dist.fillna(_age_dist.mean())

In [8]:
cluster_simil = ClusterSimilarity(n_clusters=100, gamma=1., random_state=42, sample_weight=stores_train_df[['revenue']])
similarities = cluster_simil.fit(stores_train_df[["lat", "lon"]])

def new_clustering(cluster_simil, stores_df):
    return pd.DataFrame(cluster_simil.transform(stores_df[['lat', 'lon']]), columns=cluster_simil.get_feature_names_out(), index=stores_df.store_id).reset_index()

In [9]:
kmeans = KMeans(n_clusters=500, init='k-means++')
kmeans.fit(stores_train_df[['lat', 'lon']], sample_weight=stores_train_df['revenue'])

def new_kmeans_weighted(dataframe):
    dataframe['cluster_label'] = kmeans.predict(dataframe[['lat', 'lon']])
    dataframe['cluster_label_str'] = dataframe['cluster_label'].astype(str) + '_cluster'
    dataframe.drop('cluster_label', axis=1, inplace=True)


In [10]:
# new_kmeans_weighted(stores_train_df)
# new_kmeans_weighted(stores_test_df)

In [11]:
merged_stores_train = stores_train_merged \
    .merge(grunnkrets_df, on="grunnkrets_id", how="left") \
    .merge(income_dist, on="grunnkrets_id", how="left") \
    .merge(store_closest_by_store_groups(stores_train_merged, compare_train_df, store_types), on="store_id", how="left") \
    .merge(stores_in_radius_new(stores_train_merged, compare_train_df), on="store_id", how="left") \
    .merge(new_clustering(cluster_simil, stores_train_df), on="store_id", how="left") \
    .merge(new_pop_density(stores_train_df, age_dist, grunnkrets_df, geo_groups), on="store_id", how="left") \
    .merge(average_household_income_by_geo_groups(stores_train_df, geo_groups, income_dist, household_dist, grunnkrets_df)) \
    .merge(bus_stops_distance_by_importance(stores_train_df, bus_stops_df, stop_importance_levels).reset_index(level=0), on="store_id", how="left") \
    .merge(new_age_dist(stores_train_df, age_dist, grunnkrets_df, geo_groups), on="store_id", how="left")
        
merged_stores_test = stores_test_merged \
    .merge(grunnkrets_df, on="grunnkrets_id", how="left") \
    .merge(income_dist, on="grunnkrets_id", how="left") \
    .merge(store_closest_by_store_groups(stores_test_merged, compare_test_df, store_types), on="store_id", how="left") \
    .merge(stores_in_radius_new(stores_test_merged, compare_test_df), on="store_id", how="left") \
    .merge(new_clustering(cluster_simil, stores_test_df), on="store_id", how="left") \
    .merge(new_pop_density(stores_test_df, age_dist, grunnkrets_df, geo_groups), on="store_id", how="left") \
    .merge(average_household_income_by_geo_groups(stores_test_df, geo_groups, income_dist, household_dist, grunnkrets_df)) \
    .merge(bus_stops_distance_by_importance(stores_test_df, bus_stops_df, stop_importance_levels).reset_index(level=0), on="store_id", how="left") \
    .merge(new_age_dist(stores_test_df, age_dist, grunnkrets_df, geo_groups), on="store_id", how="left") 

/var/folders/z1/l03w8mpn5xz3mghrk0j2w5gr0000gn/T/ipykernel_71538/272456682.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return population_density.fillna(population_density.mean())
/var/folders/z1/l03w8mpn5xz3mghrk0j2w5gr0000gn/T/ipykernel_71538/1837615021.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return _age_dist.fillna(_age_dist.mean())
/var/folders/z1/l03w8mpn5xz3mghrk0j2w5gr0000gn/T/ipykernel_71538/272456682.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future vers

In [12]:
target_labels = merged_stores_train[['store_id', 'revenue']].copy()
merged_stores_train = merged_stores_train.copy().drop('revenue', axis=1)

In [13]:
pd.DataFrame(merged_stores_train.columns).to_csv("features.csv")

In [84]:
inc_cols = [
    'store_id',
    'store_name',
    
    'plaace_hierarchy_id',
    'chain_name',
    'mall_name',
    'sales_channel_name_x',
    
    'grunnkrets_id',
    'municipality_name',
    't_district',
    # 'address',
    
    'lv1',
    'level2',
    'level3',
    'lv4',
]

yeo_cols = [
    'all_households',
    'avg_household_income_t_district',
    'avg_household_income_municipality_name',
    # 't_district_density',
    # 'municipality_name_density',
    
    'all_stores_in_radius',
    # 'lv1_desc_in_radius',
    'lv2_desc_in_radius',
    'lv3_desc_in_radius',
    # 'lv4_desc_in_radius',
]

box_cols = [
    'area_km2',
    # 'distance_to_lv1_desc',
    'distance_to_lv2_desc',
    'distance_to_lv3_desc',
    # 'distance_to_lv4_desc',
    # 'grunnkrets_id_density',
    # 't_district_density',
    'municipality_name_density',
    'distance_to_lokalt_knutepunkt',
    'distance_to_regionalt_knutepunkt',
    'distance_to_annen_viktig_holdeplass',
    'distance_to_nasjonalt_knutepunkt',
]

_merged_stores_train = merged_stores_train.filter(inc_cols+yeo_cols+box_cols)
_merged_stores_test = merged_stores_test.filter(inc_cols+yeo_cols+box_cols)

# drop_cols = ['geometry', 'district_name', 'grunnkrets_name', 'sales_channel_name_x', 'lv2', 'lv3', 'lat', 'lon']

# merged_stores_train.drop(drop_cols, axis=1, inplace=True)
# merged_stores_test.drop(drop_cols,axis=1, inplace=True)

In [85]:
_merged_stores_train

,store_id,store_name,plaace_hierarchy_id,chain_name,mall_name,sales_channel_name_x,grunnkrets_id,municipality_name,t_district,lv1,...,lv2_desc_in_radius,lv3_desc_in_radius,area_km2,distance_to_lv2_desc,distance_to_lv3_desc,municipality_name_density,distance_to_lokalt_knutepunkt,distance_to_regionalt_knutepunkt,distance_to_annen_viktig_holdeplass,distance_to_nasjonalt_knutepunkt
0,983540538-974187930-44774,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,MCDONALDS,Magasinet Drammen,Hamburger restaurants,6020303,Drammen,Bragernes sentrumDrammen,1,...,85,1,0.155779,6.266641e-04,4.695977e-02,6210.345317,0.005467,0.016952,0.155313,0.294614
1,987074191-973117734-44755,MCDONALD'S KLINGENBERGGATA,1.1.1.0,MCDONALDS,NaN,Hamburger restaurants,3010306,Oslo,Sentrum 3Oslo,1,...,980,19,0.264278,3.151592e-04,1.864898e-03,15685.474329,0.001257,0.001559,0.116319,0.020375
2,984890265-981157303-64491,BURGER KING HØNEFOSS,1.1.1.0,BURGER KING,Kuben Hønefoss,Hamburger restaurants,6050102,Ringerike,HønefossRingerike,1,...,33,0,0.160152,8.339069e-04,3.571459e-01,748.504200,0.025783,0.017330,0.054763,0.326362
3,914057442-992924179-126912,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,BURGER KING,Glasshuspassasjen,Hamburger restaurants,18040102,Bodø,SentrumBodø,1,...,66,1,0.095029,3.480443e-04,4.327842e-02,2752.286026,0.016854,0.000522,5.257526,0.012352
4,913018583-913063538-668469,BURGER KING TILLERTORGET,1.1.1.0,BURGER KING,Tillertorget,Hamburger restaurants,16017414,Trondheim,Tiller-HårstadTrondheim,1,...,43,1,0.251070,1.208005e-03,2.808519e-03,4496.159427,0.003370,0.644539,0.006737,0.319223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12854,915789943-915806929-781991,MEIERIGÅRDEN BRYGGERIUTSALG,2.8.11.2,NaN,NaN,Beer and soda shop,7010705,Horten,SentrumHorten,2,...,24,2,0.291337,1.679672e-03,5.398634e-03,2292.607268,0.006203,0.005978,0.009062,0.176835
12855,917921733-917982368-868081,GULATING ØLUTSALG CC GJØVIK,2.8.11.2,GULATING GRUPPEN,CC Gjøvik,Beer and soda shop,5020406,Gjøvik,SentrumGjøvik,2,...,20,1,0.137188,3.666254e-07,3.666254e-07,777.189494,0.022918,0.051408,0.168095,0.333546
12856,911721961-911764474-496764,GULATING ØLUTSALG STRØMMEN,2.8.11.2,GULATING GRUPPEN,Strømmen Storsenter,Beer and soda shop,2310803,Skedsmo,StalsbergSkedsmo,2,...,62,2,0.123431,5.596648e-04,5.596648e-04,3959.282001,0.025107,0.032679,0.037453,0.249433
12857,914337046-914343372-721294,DET GODE BRYGG,2.8.11.2,NaN,Bystasjonen,Beer and soda shop,11020113,Sandnes,SentrumSandnes,2,...,64,2,0.034857,4.838586e-04,3.165082e-03,1787.813955,0.080072,0.144443,0.101411,0.116255


In [86]:
_merged_stores_test

,store_id,store_name,plaace_hierarchy_id,chain_name,mall_name,sales_channel_name_x,grunnkrets_id,municipality_name,t_district,lv1,...,lv2_desc_in_radius,lv3_desc_in_radius,area_km2,distance_to_lv2_desc,distance_to_lv3_desc,municipality_name_density,distance_to_lokalt_knutepunkt,distance_to_regionalt_knutepunkt,distance_to_annen_viktig_holdeplass,distance_to_nasjonalt_knutepunkt
0,914206820-914239427-717245,VÅLERENGA HALAL BURGER AS,1.1.1.0,NaN,NaN,Hamburger restaurants,3012704,Oslo,VålerengaOslo,1,...,660,15,0.057027,0.000631,0.012575,109798.320300,0.010282,0.013886,0.159301,0.026472
1,916789157-916823770-824309,BURGER KING MYREN,1.1.1.0,BURGER KING,NaN,Hamburger restaurants,8061401,Skien,GulsetSkien,1,...,35,1,0.165993,0.000276,0.015989,14447.615819,0.019396,0.019258,0.135651,0.489463
2,913341082-977479363-2948,BURGER KING STOVNER,1.1.1.0,BURGER KING,Stovner Senter,Hamburger restaurants,3013917,Oslo,FossumOslo,1,...,49,2,0.236628,0.000009,0.042240,109798.320300,0.012602,0.078346,0.024432,0.171475
3,889682582-889697172-28720,BURGER KING TUNGASLETTA,1.1.1.0,BURGER KING,NaN,Hamburger restaurants,16012104,Trondheim,StrindheimTrondheim,1,...,57,0,0.983436,0.005129,0.061716,31473.115991,0.019146,0.746042,0.033637,0.409014
4,997991699-998006945-417222,VULKAN BURGERBAR,1.1.1.0,VULKAN BURGERBAR,NaN,Hamburger restaurants,3014305,Oslo,HasleOslo,1,...,679,13,0.449502,0.003988,0.012575,109798.320300,0.007982,0.017349,0.155829,0.026247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8572,917323003-917383529-844309,GULATING HAUGESUND,2.8.11.2,GULATING GRUPPEN,NaN,Beer and soda shop,11490601,Karmøy,KopervikKarmøy,2,...,4,1,1.708924,0.030498,0.030662,8974.849524,0.028465,0.121447,0.471161,0.161441
8573,917353379-917411824-845904,STAVANGER BEER AS,2.8.11.2,NaN,NaN,Beer and soda shop,11030901,Stavanger,KampenStavanger,2,...,89,4,0.310575,0.006758,0.009273,49660.900148,0.011969,0.031321,0.013542,0.014388
8574,917072302-917089248-833647,GULATING ØLUTSALG OG PUB,2.8.11.2,GULATING GRUPPEN,NaN,Beer and soda shop,2270103,Fet,HovinhøgdaFet,2,...,7,0,1.627789,0.000774,0.098197,5739.109029,0.073009,0.001239,0.103923,0.271889
8575,916960557-916993161-829908,VI BRYGGER BUTIKKDRIFT AS,2.8.11.2,NaN,NaN,Beer and soda shop,12470701,Askøy,Haugland/Davanger/RamsøyAskøy,2,...,4,0,1.103012,0.006711,0.074240,25273.860491,0.075504,0.091335,0.125114,0.160333


In [87]:
_merged_stores_train['grunnkrets_id'] = _merged_stores_train['grunnkrets_id'].astype('str')

In [88]:
_merged_stores_test['grunnkrets_id'] = _merged_stores_test['grunnkrets_id'].astype('str')

In [89]:
yeo_pipeline = make_pipeline(
    SimpleImputer(strategy="mean"),
    PowerTransformer()
)
box_pipeline = make_pipeline(
    SimpleImputer(strategy="mean"),
    PowerTransformer(method="box-cox")
)

preprocessing = make_column_transformer(
        (yeo_pipeline, yeo_cols),
        (box_pipeline, box_cols),
        remainder="passthrough"
    )

def new_transformer(merged_stores_df, preprocessing):
    return pd.DataFrame(preprocessing.transform(merged_stores_df), columns=preprocessing.get_feature_names_out(), index=merged_stores_df.index)

In [90]:
preprocessing.fit(_merged_stores_train)

_merged_stores_train = new_transformer(_merged_stores_train, preprocessing)
_merged_stores_test = new_transformer(_merged_stores_test, preprocessing)

In [91]:
pt = PowerTransformer()
rev_transformed = pt.fit_transform(target_labels[["revenue"]])
_merged_stores_train["revenue"] = rev_transformed

In [92]:
_merged_stores_train.to_csv("transformed.csv")

In [93]:
# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
train = h2o.H2OFrame(_merged_stores_train)
test = h2o.H2OFrame(_merged_stores_test)

cat_vars = inc_cols

cat_vars = [f'remainder__{i}' for i in cat_vars]

for cat in cat_vars:
    train[cat] = train[cat].asfactor()
    test[cat] = test[cat].asfactor()


#test = h2o.H2OFrame(pd.concat([test_set, stores_test_enriched[['store_id']]], axis=1).drop(drop_cols, axis=1))

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,46 mins 20 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_nwong_l2zh49
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.801 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [94]:
# Identify predictors and response
x = train.columns
y = "revenue"
x.remove(y)

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=20, seed=1, exclude_algos=['deeplearning'])
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.5564628,0.0057527,0.5522996,0.5543631,0.5530286,0.5664016,0.5562213
mean_residual_deviance,0.5384282,0.0118989,0.5255117,0.5425158,0.5425610,0.5540932,0.5274594
mse,0.5384282,0.0118989,0.5255117,0.5425158,0.5425610,0.5540932,0.5274594
null_deviance,2572.4175,88.38749,2661.035,2536.0876,2643.515,2579.3167,2442.133
r2,0.4611552,0.0158200,0.4824770,0.4468336,0.4618183,0.4448423,0.4698049
residual_deviance,1384.9482,55.233307,1376.8406,1401.8608,1422.5951,1430.1145,1293.3304
rmse,0.7337409,0.0081053,0.7249218,0.7365567,0.7365874,0.7443743,0.726264
rmsle,nan,0.0,nan,nan,nan,nan,nan


In [95]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_4_20221112_124652,0.733825,0.538499,0.556427,nan,0.538499
StackedEnsemble_BestOfFamily_1_AutoML_4_20221112_124652,0.734252,0.539126,0.556968,nan,0.539126
GBM_grid_1_AutoML_4_20221112_124652_model_2,0.748011,0.55952,0.568514,nan,0.55952
GBM_grid_1_AutoML_4_20221112_124652_model_3,0.749599,0.561899,0.569531,nan,0.561899
GBM_5_AutoML_4_20221112_124652,0.751884,0.56533,0.570047,nan,0.56533
GBM_2_AutoML_4_20221112_124652,0.754372,0.569077,0.572453,nan,0.569077
DRF_1_AutoML_4_20221112_124652,0.754812,0.569741,0.569414,nan,0.569741
GBM_3_AutoML_4_20221112_124652,0.755301,0.57048,0.573819,nan,0.57048
GBM_1_AutoML_4_20221112_124652,0.756515,0.572316,0.575658,nan,0.572316
XRT_1_AutoML_4_20221112_124652,0.758767,0.575727,0.596373,nan,0.575727


In [96]:
# The leader model is stored here
aml.leader

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.5564628,0.0057527,0.5522996,0.5543631,0.5530286,0.5664016,0.5562213
mean_residual_deviance,0.5384282,0.0118989,0.5255117,0.5425158,0.5425610,0.5540932,0.5274594
mse,0.5384282,0.0118989,0.5255117,0.5425158,0.5425610,0.5540932,0.5274594
null_deviance,2572.4175,88.38749,2661.035,2536.0876,2643.515,2579.3167,2442.133
r2,0.4611552,0.0158200,0.4824770,0.4468336,0.4618183,0.4448423,0.4698049
residual_deviance,1384.9482,55.233307,1376.8406,1401.8608,1422.5951,1430.1145,1293.3304
rmse,0.7337409,0.0081053,0.7249218,0.7365567,0.7365874,0.7443743,0.726264
rmsle,nan,0.0,nan,nan,nan,nan,nan


In [97]:
m = h2o.get_model(lb[3,"model_id"])
m.varimp(use_pandas=True).to_csv("_features2.csv")

In [82]:
preds_avg = aml.predict(test)
preds_best = aml.leader.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |

/Users/nwong/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'remainder__store_id' has levels not trained on: ["810130822-971657243-219275", "810182482-971658150-11075", "811480282-971683597-179256", "811604372-988862983-86893", "811618772-912015351-623892", "811625612-911659778-433802", "811661112-911677962-612425", "811665762-971687568-21394", "811672602-974079658-32805", "811673862-974995875-40151", ...8557 not listed..., "999649262-990192235-115914", "999649440-972257710-39400", "999653987-911550016-614706", "999655866-811552712-614673", "999657117-971794577-1512", "999662072-911698862-612468", "999665624-992886544-338735", "999666124--87921-87921", "999666728-911570688-517908", "999668615-980238296-585056"]
  warnings.warn(w)
/Users/nwong/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'remainder__store_name' has levels not trained on: ["004

███████████████████████████████████████████| (done) 100%


In [83]:
df = test.cbind(preds_best)
df = df.as_data_frame(use_pandas=True)
result = df.loc[:,("remainder__store_id", 'predict')]
submission = result.rename(columns = {"remainder__store_id": "id",  "predict" : "predicted"})
submission['predicted'] = pt.inverse_transform(submission[['predicted']])
submission.to_csv("StackedEnsembleBestOfFamily10.csv", index = False)

submission

/Users/nwong/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- predicted
Feature names seen at fit time, yet now missing:
- revenue

  warnings.warn(message, FutureWarning)


,id,predicted
0,914206820-914239427-717245,5.665496
1,916789157-916823770-824309,7.415676
2,913341082-977479363-2948,4.547560
3,889682582-889697172-28720,7.872110
4,997991699-998006945-417222,14.714318
...,...,...
8572,917323003-917383529-844309,5.254148
8573,917353379-917411824-845904,3.357050
8574,917072302-917089248-833647,4.860708
8575,916960557-916993161-829908,2.637204
